# Matt Sloane / ms9548

## Homework 12, Assignment 1 (Extra Credit)

In [31]:
# Import Modules / Setup Notebook Environment (CODE PROVIDED BY PROFESSOR)

# note this is python2 compatible only. the pykriging code is also python2 only
import os
import ast
import urllib2
import urllib
from cStringIO import StringIO
import json
import geopandas as gpd
import pandas as pd
import shapely

pui_Data = os.getenv("PUIDATA")

SQL_SOURCE = 'https://fb55.carto.com/api/v2/sql'

def queryCartoDB(query, format='CSV', source=SQL_SOURCE):
    """ queries a url with an sql compatible search
    Arguments:
    query : the sql query string
    format : the expected format
    source " the url of the API
    """
    data = urllib.urlencode({'format': format, 'q': query})
    try:
        response = urllib2.urlopen(source, data)
    except urllib2.HTTPError, e:
        raise ValueError('\n'.join(ast.literal_eval(e.readline())['error']))
    except Exception:
        raise
    return StringIO(response.read())

In [47]:
# fill in the query details in facilityQuery: 
# within a single query collect AND GROUP the data by zipcode
facilityQuery = '''SELECT COUNT (cartodb_id), zipcodes 
FROM nycasthmasdismissals
GROUP BY zipcodes
'''
facilityStream = queryCartoDB(facilityQuery, format='JSON')
facilityData   = json.loads(facilityStream.read())
facilities     = pd.DataFrame(facilityData['rows'])
facilities.columns = ['Count', 'ZIPCODE']
facilities['ZIPCODE'] = facilities['ZIPCODE'].apply(pd.to_numeric)
facilities

,Count,ZIPCODE
0,1699,10461
1,405,11373
2,1698,10451
3,220,11691
4,246,11375
5,494,10466
6,713,11212
7,167,11229
8,521,11215
9,386,11220


In [49]:
# Load in NYC Zipcodes Shapefile Data, obtained from:
# https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u
shape_path = pui_Data + "/HW12_Data/ZIP_CODE_040114.shp"
zc_shape = gpd.GeoDataFrame.from_file(shape_path)
zc_reduct = zc_shape[["ZIPCODE", "geometry"]]
zc_reduct['ZIPCODE'] = zc_reduct['ZIPCODE'].apply(pd.to_numeric)
zc_reduct

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,ZIPCODE,geometry
0,11436,"POLYGON ((1038098.251871482 188138.3800067157,..."
1,11213,"POLYGON ((1001613.712964058 186926.4395172149,..."
2,11212,"POLYGON ((1011174.275535807 183696.33770971, 1..."
3,11225,"POLYGON ((995908.3654508889 183617.6128015518,..."
4,11218,"POLYGON ((991997.1134308875 176307.4958601296,..."
5,11226,"POLYGON ((994821.4729396403 177865.7178093046,..."
6,11219,"POLYGON ((987286.3515269756 173946.4621963799,..."
7,11210,"POLYGON ((995796.0097044706 171110.0666224658,..."
8,11230,"POLYGON ((994099.2959063053 171240.7064530551,..."
9,11204,"POLYGON ((989500.1699339747 170730.1687602103,..."


In [34]:
# Convert geometry field to zip code centroid data
zc_reduct['geometry'] = gpd.GeoSeries(zc_reduct.centroid)

/opt/rh/anaconda/root/envs/PUI2016_Python2/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [51]:
zc_merge = zc_reduct.merge(facilities, on = 'ZIPCODE')

In [55]:
zc_merge

,ZIPCODE,geometry,Count
0,11213,"POLYGON ((1001613.712964058 186926.4395172149,...",376
1,11212,"POLYGON ((1011174.275535807 183696.33770971, 1...",713
2,11219,"POLYGON ((987286.3515269756 173946.4621963799,...",421
3,10466,"POLYGON ((1027197.215602726 268941.3607888818,...",494
4,10467,"POLYGON ((1017396.305742651 265444.4701448828,...",2164
5,10461,"POLYGON ((1029230.779448971 251468.9475710541,...",1699
6,10457,"POLYGON ((1011926.555855393 251674.4530655444,...",822
7,10456,"POLYGON ((1008285.147127733 245256.0794193894,...",1072
8,10451,"POLYGON ((1002809.989770979 240846.5980808884,...",1698
9,11102,"POLYGON ((1004112.849041894 222330.492068544, ...",281
